In [6]:
import setup

In [7]:
from model.yolo_module import YOLOModule
from data.pixta_face_datamodule import PIXTAFaceDataModule
import torch

In [8]:
model = torch.load(
    "../checkpoints/yolo_nano/last.ckpt", map_location=torch.device("cpu")
)

data = PIXTAFaceDataModule(
    "../datasets/pixta_face/", batch_size=2, num_workers=4, pin_memory=True
)
data.setup()

In [10]:
model['optimizer_states'][0]['param_groups'][0]['lr'] = 0.00001

In [11]:
torch.save(model, '../checkpoints/yolo_nano/last.ckpt')

In [14]:
loader = iter(data.test_dataloader())

In [15]:
from model.evaluators.postprocess import postprocess

In [16]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import ImageDraw
from PIL import ImageFont
from torchvision.transforms.functional import to_pil_image

def unnormailze(image):
    image = image.cpu().numpy()
    image = image.transpose(1, 2, 0)
    image = image * [0.2272, 0.2248, 0.2345]
    image = image + [0.6596, 0.6235, 0.5875]
    image = image * 255
    image = image.astype('uint8')
    return image

def get_sample_image(image, sample):
    # convert torch tensor to PIL image
    image = to_pil_image(unnormailze(image.cpu()))

    draw = ImageDraw.Draw(image)

    for i in range(sample['boxes'].shape[0]):
        box = sample['boxes'][i]
        x, y, w, h = int(box[0]) - int(box[2]) / 2, int(box[1]) - int(box[3]) / 2, int(box[2]), int(box[3])

        color = 'red' if sample['gender'][i] == 0 else 'blue'

        draw.rectangle(((x, y), (x+w, y+h)), outline=color, width=3)

    return image

In [17]:
def visualize_sample(image, target, pred):
    # draw in 1x2 grid
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

    ax1.imshow(get_sample_image(image, target))
    ax1.set_title('Ground Truth')
    ax1.axis('off')

    ax2.imshow(get_sample_image(image, pred))
    ax2.set_title('Prediction')
    ax2.axis('off')

    plt.show()

In [18]:
image, target = next(loader)

pred = model(image)[0]
pred = postprocess(pred, 0.2, 0.5)

print(target)

for i in range(len(pred)):
    visualize_sample(image[i], target[i], pred[i])


TypeError: 'dict' object is not callable